In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

In [2]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [3]:
def GetLaguerre(n,x):
    
    if n==0:
        LN=1
    elif n==1:
        LN=(1-x)
    else:    
        LN = ((2*n-1-x)*GetLaguerre(n-1,x)-(n-1)*GetLaguerre(n-2,x))/n
    return LN


In [4]:
print (GetLaguerre(20,x))

-46189*(1 - x)*(3 - x)/262144 + 46189*(7 - x)*(2*x/3 + (5 - x)*((1 - x)*(3 - x)/2 - 1/2)/3 - 2/3)/393216 - 46189*(11 - x)*(-8*x/15 - 4*(5 - x)*((1 - x)*(3 - x)/2 - 1/2)/15 + (9 - x)*(-3*(1 - x)*(3 - x)/8 + (7 - x)*(2*x/3 + (5 - x)*((1 - x)*(3 - x)/2 - 1/2)/3 - 2/3)/4 + 3/8)/5 + 8/15)/491520 + 46189*(15 - x)*(16*x/35 + 8*(5 - x)*((1 - x)*(3 - x)/2 - 1/2)/35 - 6*(9 - x)*(-3*(1 - x)*(3 - x)/8 + (7 - x)*(2*x/3 + (5 - x)*((1 - x)*(3 - x)/2 - 1/2)/3 - 2/3)/4 + 3/8)/35 + (13 - x)*(5*(1 - x)*(3 - x)/16 - 5*(7 - x)*(2*x/3 + (5 - x)*((1 - x)*(3 - x)/2 - 1/2)/3 - 2/3)/24 + (11 - x)*(-8*x/15 - 4*(5 - x)*((1 - x)*(3 - x)/2 - 1/2)/15 + (9 - x)*(-3*(1 - x)*(3 - x)/8 + (7 - x)*(2*x/3 + (5 - x)*((1 - x)*(3 - x)/2 - 1/2)/3 - 2/3)/4 + 3/8)/5 + 8/15)/6 - 5/16)/7 - 16/35)/573440 - 46189*(19 - x)*(-128*x/315 - 64*(5 - x)*((1 - x)*(3 - x)/2 - 1/2)/315 + 16*(9 - x)*(-3*(1 - x)*(3 - x)/8 + (7 - x)*(2*x/3 + (5 - x)*((1 - x)*(3 - x)/2 - 1/2)/3 - 2/3)/4 + 3/8)/105 - 8*(13 - x)*(5*(1 - x)*(3 - x)/16 - 5*(7 - x)*(2

In [5]:
def GetDLeguerre(n,x):
    Pn = GetLaguerre(n,x)
    return sym.diff(Pn,x,1)

In [6]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [7]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [8]:
def GetAllRootsGLeg(n):

    xn = np.linspace(0,n+(n-1)*np.sqrt(n),100)
    
    Leguerre = []
    DLeguerre = []
    
    for i in range(n+1):
        Leguerre.append(GetLaguerre(i,x))
        DLeguerre.append(GetDLeguerre(i,x))
    
    poly = sym.lambdify([x],Leguerre[n],'numpy')
    Dpoly = sym.lambdify([x],DLeguerre[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

Raices del polinomio de Grado 20:

In [9]:
print (GetAllRootsGLeg(20))

[ 0.07053989  0.37212682  0.9165821   2.74919926  4.04892531  5.61517497
  7.45901745  9.59439287 12.03880255 14.81429344 17.94889552 21.47878824
 25.45170279 29.93255463 35.01343424 40.83305706 47.61999405 55.81079575
 66.52441653]


In [10]:
def GetWeightsGLeg(n):
    Roots = GetAllRootsGLeg(n)
    Dleguerre=[]
    for i in range(n+1):
        Dleguerre.append(GetDLeguerre(i,x))
    
    Dpoly = sym.lambdify([x],Dleguerre[n],'numpy')
    Weights = Roots/(((n+1)**2)*(GetLaguerre(n+1,Roots)**2))
    
    return Weights

Pesos del polinomio de Grado 20:

In [12]:
GetWeightsGLeg(20) 

array([1.68746801e-01, 2.91254362e-01, 2.66686103e-01, 7.48260647e-02,
       2.49644173e-02, 6.20255084e-03, 1.14496239e-03, 1.55741773e-04,
       1.54014409e-05, 1.08648637e-06, 5.33012091e-08, 1.75798118e-09,
       3.72550240e-11, 4.76752925e-13, 3.37284424e-15, 1.15501434e-17,
       1.53952214e-20, 5.28644273e-24, 1.65645661e-28])